1. The idea of sample splitting to evaluate the performance of prediction ruleS is to have a single sample that is separated into at least two parts, so that each part is representative of the original sample. For our examples they are separated into training sample and test sample.

# Data

The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015. We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors; individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below 3 .

The variable of interest Y is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size n=5150.

# Data Analysis

In [166]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr

In [167]:
rdata_read = pyreadr.read_r("d:/Users/Manuela/Documents/GitHub/ECO224/Labs/data/wage2015_subsample_inference.Rdata")
data = rdata_read[ 'data' ]
type(data)
data.shape

(5150, 20)

About the structure of the data:

In [168]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5150 entries, 10 to 32643
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   wage    5150 non-null   float64 
 1   lwage   5150 non-null   float64 
 2   sex     5150 non-null   float64 
 3   shs     5150 non-null   float64 
 4   hsg     5150 non-null   float64 
 5   scl     5150 non-null   float64 
 6   clg     5150 non-null   float64 
 7   ad      5150 non-null   float64 
 8   mw      5150 non-null   float64 
 9   so      5150 non-null   float64 
 10  we      5150 non-null   float64 
 11  ne      5150 non-null   float64 
 12  exp1    5150 non-null   float64 
 13  exp2    5150 non-null   float64 
 14  exp3    5150 non-null   float64 
 15  exp4    5150 non-null   float64 
 16  occ     5150 non-null   category
 17  occ2    5150 non-null   category
 18  ind     5150 non-null   category
 19  ind2    5150 non-null   category
dtypes: category(4), float64(16)
memory usage: 736.3+ KB


In [169]:
data.describe()

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4
count,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000
mean,23.410410,2.970787,0.444466,0.023301,0.243883,0.278058,0.317670,0.137087,0.259612,0.296505,0.216117,0.227767,13.760583,3.018925,8.235867,25.118038
std,21.003016,0.570385,0.496955,0.150872,0.429465,0.448086,0.465616,0.343973,0.438464,0.456761,0.411635,0.419432,10.609465,4.000904,14.488962,53.530225
min,3.021978,1.105912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.461538,2.599837,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.250000,0.125000,0.062500
50%,19.230769,2.956512,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,1.000000,1.000000
75%,27.777778,3.324236,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,21.000000,4.410000,9.261000,19.448100
max,528.845673,6.270697,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,22.090000,103.823000,487.968100


We are constructing the output variable $Y$ and the matrix $Z$ which includes the characteristics of workers that are given in the data.

In [172]:
Y = np.log2(data['wage']) 
n = len(Y)
z = data.loc[:, ~data.columns.isin(['wage', 'lwage','Unnamed: 0'])]
p = z.shape[1]

print("Number of observation:", n, '\n')
print( "Number of raw regressors:", p)

Number of observation: 5150 

Number of raw regressors: 18


For the outcome variable wage and a subset of the raw regressors, we calculate the empirical mean:

In [173]:
Z_scl = data[data[ 'scl' ] == 0 ]
Z_clg = data[data[ 'clg' ] == 0 ]
Z_data = pd.concat([Z_scl,Z_clg])

Z_subset = data.loc[:, data.columns.isin(["lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"])]
table = Z_subset.mean(axis=0)
table

lwage     2.970787
sex       0.444466
shs       0.023301
hsg       0.243883
scl       0.278058
clg       0.317670
ad        0.137087
mw        0.259612
so        0.296505
we        0.216117
ne        0.227767
exp1     13.760583
dtype: float64

In [174]:
table = pd.DataFrame(data=table, columns={"Sample mean":"0"} )
table.index
index1 = list(table.index)
index2 = ["Log Wage","Sex","Some High School","High School Graduate",\
          "Some College","College Graduate", "Advanced Degree","Midwest",\
          "South","West","Northeast","Experience"]

In [175]:
table = table.rename(index=dict(zip(index1,index2)))
table

,Sample mean
Log Wage,2.970787
Sex,0.444466
Some High School,0.023301
High School Graduate,0.243883
Some College,0.278058
College Graduate,0.317670
Advanced Degree,0.137087
Midwest,0.259612
South,0.296505
West,0.216117


Here we can see the share of female workers in this sample is ~44% ($sex=1$ if female) and for th experience es ~13.76%.

# Predicted

We will construct a prediction rule for hourly wage $Y$, which depends linearly on job-relevant characteristics $X$:

$$\label{decompose}
Y = \beta'X+ \epsilon.
$$
Our goals are

Predict wages using various characteristics of workers.

Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.

We employ two different specifications for prediction:

Basic Model: $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators, occupation and industry indicators, regional indicators).
Flexible Model: $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g., ${exp}^2$ and ${exp}^3$) and additional two-way interactions of polynomial in experience with other regressors.
Using the Flexible Model, enables us to approximate the real relationship by a more complex regression model and therefore to reduce the bias. This Model increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, we are fit both models to our data by running ordinary least squares (ols):

In [176]:
# Import packages for OLS regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [177]:
# 1. basic model
basic = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
basic_results = smf.ols(basic , data=Z_data).fit()
print(basic_results.summary()) # estimated coefficients
print( "Number of regressors in the basic model:",len(basic_results.params), '\n')  # number of regressors in the Basic Model

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.329
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     70.27
Date:                Fri, 17 Sep 2021   Prob (F-statistic):               0.00
Time:                        15:44:50   Log-Likelihood:                -4888.7
No. Observations:                7232   AIC:                             9879.
Df Residuals:                    7181   BIC:                         1.023e+04
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.5406      0.045     78.794      0.0

In [178]:
# 2. flexible model
flex = 'lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'
flex_results_0 = smf.ols(flex , data=Z_data)
flex_results = smf.ols(flex , data=Z_data).fit()
print(flex_results.summary()) # estimated coefficients
print( "Number of regressors in the flexible model:",len(flex_results.params), '\n') # number of regressors in the Flexible Model

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     7.353
Date:                Fri, 17 Sep 2021   Prob (F-statistic):               0.00
Time:                        15:44:59   Log-Likelihood:                -4033.7
No. Observations:                7232   AIC:                             9625.
Df Residuals:                    6453   BIC:                         1.499e+04
Df Model:                         778                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 2.79

We can observe that the models have different numbers of regressors: the basic model consists of 51 regressors, while the flexible model consists of 979 regressors.

## Lasso Regression

In [179]:
# Import relevant packages for lasso 
from sklearn.linear_model import LassoCV
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [180]:
# Get exogenous variables from flexible model
X = flex_results_0.exog
X.shape

# Set endogenous variable
lwage = Z_data["lwage"]
lwage.shape

(7232,)

In [187]:
alpha=0.1
alpha_1=0.6
alpha_2=0.65

In [182]:
# Set penalty value = 0.1
#reg = linear_model.Lasso(alpha=0.1/np.log(len(lwage)))
reg = linear_model.Lasso(alpha = alpha)

# LASSO regression for flexible model
reg.fit(X, lwage)
lwage_lasso_fitted = reg.fit(X, lwage).predict( X )

# coefficients 
reg.coef_
print('Lasso Regression: R^2 score', reg.score(X, lwage))

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 357.71768748352656, tolerance: 0.2437264828580145
  model = cd_fast.enet_coordinate_descent(


Lasso Regression: R^2 score 0.19053431255708364


C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 357.71768748352656, tolerance: 0.2437264828580145
  model = cd_fast.enet_coordinate_descent(


In [188]:
# Set penalty value = 0.6
reg1 = linear_model.Lasso(alpha = alpha_1)

# LASSO regression for flexible model
reg1.fit(X, lwage)
lwage_lasso_fitted1 = reg1.fit(X, lwage).predict( X )

# coefficients 
reg1.coef_
print('Lasso Regression: R^2 score', reg1.score(X, lwage))

Lasso Regression: R^2 score 0.05190007110917727


In [189]:
# Set penalty value = 0.65
reg2 = linear_model.Lasso(alpha = alpha_2)

# LASSO regression for flexible model
reg2.fit(X, lwage)
lwage_lasso_fitted2 = reg2.fit(X, lwage).predict( X )

# coefficients 
reg2.coef_
print('Lasso Regression: R^2 score', reg2.score(X, lwage))

Lasso Regression: R^2 score 0.04678942276776776


We can observe that, with an alpha of 0.1, the Lasso model presents convergence problems, so it is necessary to increase the value of this alpha to be penalized more. So it has been found that from the value 0.6 the model does not present convergence problems, then it has been tested with a more accurate value of 0.65, which matches correctly. Thus, it is also shown that the value of R^2 decreases as the value of alpha is higher.

In [190]:
# Check predicted values
lwage_lasso_fitted

array([3.01766521, 3.33892844, 2.78461611, ..., 3.06897067, 2.79801705,
       3.22982614])

Then, we evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [191]:
# Basic Model
basic = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
basic_results = smf.ols(basic , data=Z_data).fit()

# Flexible model 
flex = 'lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'
flex_results = smf.ols(flex , data=Z_data).fit()

In [192]:
# Assess the predictive performance
R2_1 = basic_results.rsquared
print("R-squared for the basic model: ", R2_1, "\n")
R2_adj1 = basic_results.rsquared_adj
print("adjusted R-squared for the basic model: ", R2_adj1, "\n")


R2_2 = flex_results.rsquared
print("R-squared for the basic model: ", R2_2, "\n")
R2_adj2 = flex_results.rsquared_adj
print("adjusted R-squared for the basic model: ", R2_adj2, "\n")

R2_L = reg.score(flex_results_0.exog, lwage)
print("R-squared for LASSO: ", R2_L, "\n")
R2_adjL = 1 - (1-R2_L)*(len(lwage)-1)/(len(lwage)-X.shape[1]-1)
print("adjusted R-squared for LASSO: ", R2_adjL, "\n")

R-squared for the basic model:  0.32852829994094634 

adjusted R-squared for the basic model:  0.3238529643326812 

R-squared for the basic model:  0.46992524337652 

adjusted R-squared for the basic model:  0.4060172686898521 

R-squared for LASSO:  0.19053431255708364 

adjusted R-squared for LASSO:  0.0637801686020909 



In [193]:
# calculating the MSE
MSE1 =  np.mean(basic_results.resid**2)
print("MSE for the basic model: ", MSE1, "\n")
p1 = len(basic_results.params) # number of regressors
n = len(lwage)
MSE_adj1  = (n/(n-p1))*MSE1
print("adjusted MSE for the basic model: ", MSE_adj1, "\n")

MSE2 =  np.mean(flex_results.resid**2)
print("MSE for the flexible model: ", MSE2, "\n")
p2 = len(flex_results.params) # number of regressors
n = len(lwage)
MSE_adj2  = (n/(n-p2))*MSE2
print("adjusted MSE for the flexible model: ", MSE_adj2, "\n")


MSEL = mean_squared_error(lwage, lwage_lasso_fitted)
print("MSE for the LASSO model: ", MSEL, "\n")
pL = reg.coef_.shape[0] # number of regressors
n = len(lwage)
MSE_adjL  = (n/(n-pL))*MSEL
print("adjusted MSE for LASSO model: ", MSE_adjL, "\n")

MSE for the basic model:  0.22629346763562672 

adjusted MSE for the basic model:  0.22790062079666512 

MSE for the flexible model:  0.17864111737231547 

adjusted MSE for the flexible model:  0.20661003691613392 

MSE for the LASSO model:  0.2727989836763092 

adjusted MSE for LASSO model:  0.3155097153281734 



In [143]:
pip install array_to_latex

Note: you may need to restart the kernel to use updated packages.


In [194]:
# Package for latex table 
import array_to_latex as a2l

table = np.zeros((3, 5))
table[0,0:5] = [p1, R2_1, MSE1, R2_adj1, MSE_adj1]
table[1,0:5] = [p2, R2_2, MSE2, R2_adj2, MSE_adj2]
table[2,0:5] = [pL, R2_L, MSEL, R2_adjL, MSE_adjL]
table

array([[5.10000000e+01, 3.28528300e-01, 2.26293468e-01, 3.23852964e-01,
        2.27900621e-01],
       [9.79000000e+02, 4.69925243e-01, 1.78641117e-01, 4.06017269e-01,
        2.06610037e-01],
       [9.79000000e+02, 1.90534313e-01, 2.72798984e-01, 6.37801686e-02,
        3.15509715e-01]])

In [195]:
table = pd.DataFrame(table, columns = ["p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$"], \
                      index = ["basic reg","flexible reg", "lasso flex"])
table

,p,$R^2_{sample}$,$MSE_{sample}$,$R^2_{adjusted}$,$MSE_{adjusted}$
basic reg,51.0,0.328528,0.226293,0.323853,0.227901
flexible reg,979.0,0.469925,0.178641,0.406017,0.206610
lasso flex,979.0,0.190534,0.272799,0.063780,0.315510


Considering all measures above, the flexible model performs slightly better than the basic model.

## Data Splitting

Measure the prediction quality of the two models via data splitting:

Randomly split the data into one training sample and one testing sample. Here we just use a simple method.
Use the training sample for estimating the parameters of the Basic Model and the Flexible Model.
Use the testing sample for evaluation. Predict the $\mathtt{wage}$ of every observation in the testing sample based on the estimated parameters in the training sample.
Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models.

In [196]:
# Import relevant packages for splitting data
import random
import math

# Set Seed
# to make the results replicable (generating random numbers)
np.random.seed(0)
random = np.random.randint(0,n, size=math.floor(n))
Z_data["random"] = random
random    # the array does not change

array([2732, 2607, 1653, ..., 2518, 2428, 4618])

In [197]:
data_2 = Z_data.sort_values(by=['random'])
data_2.head()

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,...,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2,random
rownames,,,,,,,,,,,,,,,,,,,,,
6274,9.615385,2.263364,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,7.0,0.49,0.343,0.2401,5620,17,4970,9,0
4180,12.500000,2.525729,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,26.0,6.76,17.576,45.6976,5120,17,7790,16,0
29092,36.858974,3.607099,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,29.0,8.41,24.389,70.7281,230,1,7860,17,0
1748,15.384615,2.733368,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,25.0,6.25,15.625,39.0625,2900,9,6590,11,2
32548,24.475524,3.197674,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,24.0,5.76,13.824,33.1776,2310,8,7860,17,2


In [198]:
# Create training and testing sample 
train = data_2[ : math.floor(n*4/5)]    # training sample
test =  data_2[ math.floor(n*4/5) : ]   # testing sample
print(train.shape)
print(test.shape)

(5785, 21)
(1447, 21)


In [199]:
# Basic Model
basic = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
basic_results = smf.ols(basic , data=Z_data).fit()

# Flexible model 
flex = 'lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'
flex_results = smf.ols(flex , data=Z_data).fit()

In [200]:
# basic model
# estimating the parameters in the training sample
basic_results = smf.ols(basic , data=train).fit()
print(basic_results.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.321
Method:                 Least Squares   F-statistic:                     55.78
Date:                Fri, 17 Sep 2021   Prob (F-statistic):               0.00
Time:                        15:48:10   Log-Likelihood:                -3892.5
No. Observations:                5785   AIC:                             7887.
Df Residuals:                    5734   BIC:                             8227.
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.5582      0.050     70.593      0.0

In [201]:
lwage_test = test["lwage"].values
#test = test.drop(columns=['wage', 'lwage', 'random'])
#test

In [202]:
# calculating the out-of-sample MSE
test = sm.add_constant(test)   #add constant 

lwage_pred =  basic_results.predict(test) # predict out of sample
print(lwage_pred)

rownames
17518    2.637080
7461     2.537063
16082    3.583619
1947     3.537958
6082     2.388421
           ...   
15958    3.588832
20744    2.666421
1306     3.042807
9206     2.929551
27620    2.815704
Length: 1447, dtype: float64


In [203]:
MSE_test1 = np.sum((lwage_test-lwage_pred)**2)/len(lwage_test)
R2_test1  = 1 - MSE_test1/np.var(lwage_test)

print("Test MSE for the basic model: ", MSE_test1, " ")
print("Test R2 for the basic model: ", R2_test1)

Test MSE for the basic model:  0.23392402364335554  
Test R2 for the basic model:  0.3275283611021428


In the basic model, the $MSE_{test}$ is quite closed to the $MSE_{sample}$.

In [204]:
# Flexible model
# estimating the parameters in the training sample
flex_results = smf.ols(flex , data=train).fit()

# calculating the out-of-sample MSE
lwage_flex_pred =  flex_results.predict(test) # predict out of sample
lwage_test = test["lwage"].values

MSE_test2 = np.sum((lwage_test-lwage_flex_pred)**2)/len(lwage_test)
R2_test2  = 1 - MSE_test2/np.var(lwage_test)

print("Test MSE for the flexible model: ", MSE_test2, " ")
print("Test R2 for the flexible model: ", R2_test2)

Test MSE for the flexible model:  0.22838942838980827  
Test R2 for the flexible model:  0.3434389045462126


In the flexible model, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is appreciably large.

It is worth to notice that the $MSE_{test}$ vary across different data splits. Hence, it is a good idea average the out-of-sample MSE over different data splits to get valid results. Nevertheless, we observe that, based on the out-of-sample $MSE$, the basic model using ols regression performs is about as well (or slightly better) than the flexible model.

We are goin to use lasso regression (least absolute shrinkage and selection operator) in the flexible model instead of ols regression because is a penalized regression method that can be used to reduce the complexity of a regression model when the number of regressors $p$ is relatively large in relation to $n$.

Note that the out-of-sample $MSE$ on the test sample can be computed for any other black-box prediction method as well. So, we are finally compare the performance of lasso regression in the flexible model to ols regression.

In [205]:
# flexible model using lasso
# get exogenous variables from training data used in flex model
flex_results_0 = smf.ols(flex , data=train)
X_train = flex_results_0.exog
print(X_train.shape)

# Get endogenous variable 
lwage_train = train["lwage"]
print(lwage_train.shape)

(5785, 979)
(5785,)


In [206]:
# flexible model using lasso
# get exogenous variables from testing data used in flex model
flex_results_1 = smf.ols(flex , data=test)
X_test = flex_results_1.exog
print(X_test.shape)

# Get endogenous variable 
lwage_test = test["lwage"]
print(lwage_test.shape)

(1447, 979)
(1447,)


In [207]:
# calculating the out-of-sample MSE
reg = linear_model.Lasso(alpha=0.1)
lwage_lasso_fitted = reg.fit(X_train, lwage_train).predict( X_test )

MSE_lasso = np.sum((lwage_test-lwage_lasso_fitted)**2)/len(lwage_test)
R2_lasso  = 1 - MSE_lasso/np.var(lwage_test)

print("Test MSE for the flexible model: ", MSE_lasso, " ")
print("Test R2 for the flexible model: ", R2_lasso)

Test MSE for the flexible model:  0.2821326513894069  
Test R2 for the flexible model:  0.18894090691731513


C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 335.97419631154617, tolerance: 0.19337855361002732
  model = cd_fast.enet_coordinate_descent(


In [211]:
# calculating the out-of-sample MSE
reg2 = linear_model.Lasso(alpha=0.5)
lwage_lasso_fitted2 = reg2.fit(X_train, lwage_train).predict( X_test )

MSE_lasso_2 = np.sum((lwage_test-lwage_lasso_fitted2)**2)/len(lwage_test)
R2_lasso_2  = 1 - MSE_lasso_2/np.var(lwage_test)

print("Test MSE for the flexible model with alpha = 0.3: ", MSE_lasso_2, " ")
print("Test R2 for the flexible model with alpha = 0.3: ", R2_lasso_2)

Test MSE for the flexible model with alpha = 0.3:  0.3280405416425374  
Test R2 for the flexible model with alpha = 0.3:  0.05696748359789916


Summarizing the results:

In [212]:
# Package for latex table 
import array_to_latex as a2l

table2 = np.zeros((3, 2))
table2[0,0] = MSE_test1
table2[1,0] = MSE_test2
table2[2,0] = MSE_lasso_2
table2[0,1] = R2_test1
table2[1,1] = R2_test2
table2[2,1] = R2_lasso_2

table2 = pd.DataFrame(table2, columns = ["$MSE_{test}$", "$R^2_{test}$"], \
                      index = ["basic reg","flexible reg","lasso regression"])
table2

,$MSE_{test}$,$R^2_{test}$
basic reg,0.233924,0.327528
flexible reg,0.228389,0.343439
lasso regression,0.328041,0.056967


In [213]:
table2.to_latex
print(table2.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  \$MSE\_\{test\}\$ &  \$R\textasciicircum 2\_\{test\}\$ \\
\midrule
basic reg        &      0.233924 &      0.327528 \\
flexible reg     &      0.228389 &      0.343439 \\
lasso regression &      0.328041 &      0.056967 \\
\bottomrule
\end{tabular}



Even when the penalty has been increased for the model using Lasso, it is obtained that the flexible model is the one with the best performance, since the value of the MSE of the evaluation sample is lower, together with a high value of its R^2.

## Two new cases of Partialling-Out using Lasso

Y = log(wage), D = sex

### Case 1: Partialling-Out using lasso 1 :
#### Matrix W = 'exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'

In [214]:
# models
# model for Y
flex_y = 'lwage ~  (exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2)'
# model for D
flex_d = 'sex ~ (exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2)' 

# partialling-out the linear effect of W from Y
t_Y = smf.ols( formula = flex_y , data = Z_data ).fit().resid

# partialling-out the linear effect of W from D
t_D = smf.ols( formula = flex_d , data = Z_data ).fit().resid

data_res = pd.DataFrame( np.vstack(( t_Y.values , t_D.values )).T , columns = [ 't_Y', 't_D' ] )
# regression of Y on D after partialling-out the effect of W
partial_fit =  smf.ols( formula = 't_Y ~ t_D' , data = data_res ).fit()
partial_est = partial_fit.summary2().tables[1]['Coef.']['t_D']

print("Coefficient for D via partialling-out", partial_est)

# standard error
HCV_coefs = partial_fit.cov_HC0
partial_se = np.power( HCV_coefs.diagonal() , 0.5)[1]

# confidence interval
partial_fit.conf_int( alpha=0.05 ).iloc[1, :]

Coefficient for D via partialling-out -0.07961023928363375


0   -0.104635
1   -0.054585
Name: t_D, dtype: float64

### Case 2: Partialling-Out using lasso 2 :
#### Matrix W = (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'

In [215]:
# models
# model for Y
flex_y = 'lwage ~  ((exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2)'
# model for D
flex_d = 'sex ~ ((exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2)' 

# partialling-out the linear effect of W from Y
t_Y = smf.ols( formula = flex_y , data = Z_data ).fit().resid

# partialling-out the linear effect of W from D
t_D = smf.ols( formula = flex_d , data = Z_data ).fit().resid

data_res = pd.DataFrame( np.vstack(( t_Y.values , t_D.values )).T , columns = [ 't_Y', 't_D' ] )
# regression of Y on D after partialling-out the effect of W
partial_fit =  smf.ols( formula = 't_Y ~ t_D' , data = data_res ).fit()
partial_est = partial_fit.summary2().tables[1]['Coef.']['t_D']

print("Coefficient for D via partialling-out", partial_est)

# standard error
HCV_coefs = partial_fit.cov_HC0
partial_se = np.power( HCV_coefs.diagonal() , 0.5)[1]

# confidence interval
partial_fit.conf_int( alpha=0.05 ).iloc[1, :]

Coefficient for D via partialling-out -0.06760692571926313


0   -0.092272
1   -0.042942
Name: t_D, dtype: float64